In [18]:
import pandas as pd
import numpy as np
import os
import csv
import ast
from collections import Counter
import geopandas as gpd
import h5py
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys

In [19]:
cluster_name = {0: 'Glocery&Pharmacies', 1: 'Retails', 2: 'Arts&Entertainment', 3: 'Restaurants&Bars',
                4: 'Educations', 5: 'Healthcares', 6: 'others'}

In [20]:
###1 mobility
mobility_map_method = '1to1'
#  loss_visitors
#  1tomore
#  1to1

In [21]:
df_flow = pd.read_csv('../data/raw_mobility/outflow_7c_wk_raw_{}.csv'.format(mobility_map_method))
df_flow['MODZCTA'] = df_flow['MODZCTA'].astype(str)

In [22]:
cluster_cols = [cluster_name[0]+'_{}'.format(i) for i in range(56)] 
# 56 weeks from 2020-02-04 to 2021-03-02
# only need 53 weeks from 2020-02-24 to 2021-02-02
df_vist_cluster_i = df_flow[['MODZCTA','mod_idx']+cluster_cols].copy()

final_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA','mod_idx'], value_vars = cluster_cols[3:3+51], var_name='week', value_name=cluster_name[0]+'_visits_weekly')
#### 
# Correctly adjust the code to perform the intended replacement
final_df['week'] = final_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))

for c_idx in range(1,7):
    print(cluster_name[c_idx])
    cluster_cols = [cluster_name[c_idx]+'_{}'.format(i) for i in range(56)]
    df_vist_cluster_i = df_flow[['MODZCTA', 'mod_idx']+cluster_cols].copy()
    temp_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA', 'mod_idx'], value_vars = cluster_cols[3:3+51], 
                    var_name='week', value_name=cluster_name[c_idx]+'_visits_weekly')
    temp_df['week'] = temp_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))
    final_df = pd.merge(final_df, temp_df, on=['MODZCTA','mod_idx','week'])

Retails
Arts&Entertainment
Restaurants&Bars
Educations
Healthcares
others


Merge all the other data at the zipcode level

In [23]:
### 2 read scores data at zipcode level#########
final_df['MODZCTA'] = final_df['MODZCTA'].astype(int)
df_mod_score = pd.read_csv('../results/scores/modzcta_scores.csv')
df_2 = final_df.merge(df_mod_score, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_2.dropna(subset=['score_median'],inplace=True)
print(len(df_2))

6477


In [24]:
##### 3 social economic data ######
df_sed_nyc = pd.read_csv('../Data/sed_nyc_modzcta_2021.csv')
# df_sed_nyc = pd.read_csv('../Data/sed_old.csv')
# df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='right')
### when merged on the right will have the subset of zipcodes 
##  where the zipcodes counts are satisfied
print(len(df_3))

816


In [25]:
### 4 merge with borough cases ###
df_borough = pd.read_csv('../Data/mappings/data-by-modzcta.csv')
df_borough['borough'] = df_borough['BOROUGH_GROUP'].replace({'Manhattan':'MN', 'Brooklyn':'BK', 'Queens':'QN', 'Bronx':'BX', 'Staten Island':'SI'})
df_borough_s = df_borough.rename(columns={'MODIFIED_ZCTA':'MODZCTA',
                                          'COVID_CONFIRMED_CASE_COUNT':'COVID_CONFIRMED_CASE_COUNT_final', 
                                          'COVID_PROBABLE_CASE_COUNT':'COVID_PROBABLE_CASE_COUNT_final',
                                          'COVID_CASE_COUNT':'COVID_CASE_COUNT_final',
                                          'COVID_CONFIRMED_CASE_RATE':'COVID_CONFIRMED_CASE_RATE_final', 
                                          'COVID_CASE_RATE':'COVID_CASE_RATE_final',
                                          'COVID_DEATH_COUNT':'COVID_DEATH_COUNT_final', 
                                          'COVID_DEATH_RATE':'COVID_DEATH_RATE_final' })
df_3 = df_3.merge(df_borough_s, on='MODZCTA',how='left')

In [27]:
covid_cases = pd.read_csv('../data/cases_borough_weekly.csv')
for col in ['CASE_COUNT','DEATH_COUNT', 'borough_case_count', 'borough_death_count']:
    covid_cases[col] = covid_cases[col].astype(float)
    covid_cases.loc[covid_cases[col]==0,col] = 0.1
    covid_cases[col+'_log'] = np.log(covid_cases[col])

df_4 = df_3.merge(covid_cases, on=['borough', 'week'], how='left')
print(len(df_4))

816


In [12]:
### 5 merge with policies ####
df_policies = pd.read_csv('../data/stringency_index_ny_wk.csv')
visits_scores_wk = df_4.merge(df_policies, on='week', how='left')
print(len(visits_scores_wk))

816


In [11]:
visits_scores_wk.to_csv('../data/unpivot_merged_data_raw_{}_s.csv'.format(mobility_map_method),index=False)

In [14]:
visits_scores_wk.columns

Index(['MODZCTA', 'mod_idx', 'week', 'Glocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly', 'others_visits_weekly', 'score_mean',
       'score_median', 'score_std', 'gain_bias_mean', 'gain_bias_median',
       'gain_bias_std', 'loss_bias_mean', 'loss_bias_median', 'loss_bias_std',
       'loss_aversion_scores_mean', 'loss_aversion_scores_median',
       'loss_aversion_scores_std', 'rescale_avail_mean',
       'rescale_avail_median', 'rescale_avail_std', 'rescale_realistic_mean',
       'rescale_realistic_median', 'rescale_realistic_std', 'agency_mean',
       'agency_median', 'agency_std', 'regulated_tdscores_mean',
       'regulated_tdscores_median', 'regulated_tdscores_std',
       'regulated_agency_mean', 'regulated_agency_median',
       'regulated_agency_std', 'regulated_loss_mean', 'regulated_loss_median',
       'regulated_los

In [16]:
for col in ['Glocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly']:
       visits_scores_wk[col+'_pp'] = visits_scores_wk[col]/visits_scores_wk['Population']*100

In [17]:
visits_scores_wk['Bachelor_percent'] = visits_scores_wk['Bachelor']/visits_scores_wk['Population']

In [ ]:
select_cols = ['Glocery&Pharmacies_visits_weekly_pp',
       'Retails_visits_weekly_pp', 'Arts&Entertainment_visits_weekly_pp',
       'Restaurants&Bars_visits_weekly_pp', 'Educations_visits_weekly_pp',
       'Healthcares_visits_weekly_pp', 
       'regulated_tdscores_median', 
       'regulated_agency_median',
       'regulated_loss_median',
       'weighted_estimated_average_age',
       'Household_income', 
       'Bachelor_percent',
       'Household_num', 
       'No_health_insurance_rate', 'No_vehicles_rate',
       'borough_case_count_log',
       'StringencyIndex_WeightedAverage']

In [ ]:
from matplotlib import colors
from sklearn.preprocessing import StandardScaler

# Assuming gdf_filtered is your GeoDataFrame and select_cols is a list of column names
scaler = StandardScaler()

# Fit the scaler to the data and transform it
gdf_normalized = scaler.fit_transform(gdf_filtered[select_cols])

# Convert the normalized array back to a DataFrame
df_normalized = pd.DataFrame(gdf_normalized, columns=select_cols)

# Now compute the correlation on the normalized DataFrame
corr = df_normalized.corr()

# corr = gdf_filtered[select_cols].corr()
fig, ax = plt.subplots(figsize=(15, 15))  # Optional: specifies the figure size
norm = colors.TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)

cax = ax.matshow(corr, cmap='coolwarm',norm=norm)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)

# Add a color bar
fig.colorbar(cax)

# Optional: Add annotations with the correlation values
for (i, j), val in np.ndenumerate(corr):
    ax.text(j, i, f'{val:.2f}', ha='center', va='center', color='white')

plt.title("Correlation Matrix Heatmap", pad=20)
plt.show()